In [4]:
import pandas as pd
import trino
import json

# Cấu hình để Pandas hiển thị đầy đủ nội dung (quan trọng khi xem JSON)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)

print("Các thư viện đã sẵn sàng.")

Các thư viện đã sẵn sàng.


In [5]:
# Kết nối tới Trino service
conn = trino.dbapi.connect(
    host='localhost',
    port=8080,
    user='python-client', # Tên user bất kỳ
    catalog='lakehouse', # Tên catalog Iceberg bạn định nghĩa trong Flink
    schema='rva'         # Tên database bạn định nghĩa trong Flink
)

cursor = conn.cursor()
print(f"Kết nối Trino (Catalog: {conn.catalog}, Schema: {conn.schema}) thành công!")

Kết nối Trino (Catalog: lakehouse, Schema: rva) thành công!


In [6]:
cursor.execute("SHOW TABLES")
tables = cursor.fetchall()
print("Các bảng có trong schema 'rva':")
print(tables)

# Kết quả mong đợi: [('bronze_raw',)]

Các bảng có trong schema 'rva':
[['bronze_raw']]


In [7]:
# Xem schema của bảng bronze_raw
df_schema = pd.read_sql("DESCRIBE bronze_raw", conn)
df_schema

/tmp/ipykernel_58481/1661251802.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_schema = pd.read_sql("DESCRIBE bronze_raw", conn)


,Column,Type,Extra,Comment
0,schema_version,varchar,,
1,pipeline_run_id,varchar,,
2,frame_index,bigint,,
3,payload,varchar,,
4,camera_id,varchar,,
5,store_id,varchar,,
6,ingest_ts,timestamp(6) with time zone,,


In [13]:
# Đếm tổng số dòng (records) trong bảng
# Con số này sẽ tăng lên nếu Flink job của bạn vẫn đang chạy
cursor.execute("SELECT COUNT(*) FROM bronze_raw")
total_rows = cursor.fetchone()[0]

print(f"Tổng số dòng (records) trong bảng 'bronze_raw': {total_rows}")

Tổng số dòng (records) trong bảng 'bronze_raw': 410


In [14]:
# Tìm ingest_ts (thời gian ghi dữ liệu) cũ nhất và mới nhất
df_time_range = pd.read_sql("""
    SELECT 
        MIN(ingest_ts) AS min_timestamp,
        MAX(ingest_ts) AS max_timestamp
    FROM bronze_raw
""", conn)

print("Phạm vi thời gian của dữ liệu:")
df_time_range

Phạm vi thời gian của dữ liệu:


/tmp/ipykernel_58481/4236476250.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_time_range = pd.read_sql("""


,min_timestamp,max_timestamp
0,2025-11-06 09:52:52.096000+00:00,2025-11-06 09:53:02.525000+00:00


In [8]:
# Đọc 5 dòng đầu tiên
df_sample = pd.read_sql("SELECT * FROM bronze_raw LIMIT 5", conn)
df_sample

/tmp/ipykernel_58481/3061181187.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sample = pd.read_sql("SELECT * FROM bronze_raw LIMIT 5", conn)


,schema_version,pipeline_run_id,frame_index,payload,camera_id,store_id,ingest_ts
0,1.0,27f0344f42da49558518258a3d50263c,1,"{""schema_version"": ""1.0"", ""pipeline_run_id"": ""27f0344f42da49558518258a3d50263c"", ""source"": {""store_id"": ""store_01"", ""camera_id"": ""cam_01"", ""stream_id"": ""stream_01""}, ""frame_index"": 1, ""capture_ts"": ""2025-10-31T13:22:36.674047+00:00"", ""image_size"": {""width"": 1280, ""height"": 720}, ""detections"": [{""det_id"": ""1-0"", ""class"": null, ""class_id"": 0, ""conf"": 0.8199, ""bbox"": {""x1"": 345.0, ""y1"": 58.0, ""x2"": 438.0, ""y2"": 430.0}, ""bbox_norm"": {""x"": 0.26953125, ""y"": 0.08055555555555556, ""w"": 0.07265625, ""h"": 0.5166666666666667}, ""centroid"": {""x"": 392, ""y"": 244}, ""centroid_norm"": {""x"": 0.305859375, ""y"": 0.3388888888888889}, ""track_id"": null}]}",cam_01,store_01,2025-11-06 09:52:52.096000+00:00
1,1.0,27f0344f42da49558518258a3d50263c,2,"{""schema_version"": ""1.0"", ""pipeline_run_id"": ""27f0344f42da49558518258a3d50263c"", ""source"": {""store_id"": ""store_01"", ""camera_id"": ""cam_01"", ""stream_id"": ""stream_01""}, ""frame_index"": 2, ""capture_ts"": ""2025-10-31T13:22:36.712082+00:00"", ""image_size"": {""width"": 1280, ""height"": 720}, ""detections"": [{""det_id"": ""2-0"", ""class"": null, ""class_id"": 0, ""conf"": 0.8254, ""bbox"": {""x1"": 344.0, ""y1"": 58.0, ""x2"": 438.0, ""y2"": 430.0}, ""bbox_norm"": {""x"": 0.26875, ""y"": 0.08055555555555556, ""w"": 0.0734375, ""h"": 0.5166666666666667}, ""centroid"": {""x"": 391, ""y"": 244}, ""centroid_norm"": {""x"": 0.30546875, ""y"": 0.3388888888888889}, ""track_id"": null}]}",cam_01,store_01,2025-11-06 09:52:52.494000+00:00
2,1.0,27f0344f42da49558518258a3d50263c,3,"{""schema_version"": ""1.0"", ""pipeline_run_id"": ""27f0344f42da49558518258a3d50263c"", ""source"": {""store_id"": ""store_01"", ""camera_id"": ""cam_01"", ""stream_id"": ""stream_01""}, ""frame_index"": 3, ""capture_ts"": ""2025-10-31T13:22:36.750969+00:00"", ""image_size"": {""width"": 1280, ""height"": 720}, ""detections"": [{""det_id"": ""3-0"", ""class"": null, ""class_id"": 0, ""conf"": 0.8267, ""bbox"": {""x1"": 344.0, ""y1"": 58.0, ""x2"": 438.0, ""y2"": 429.0}, ""bbox_norm"": {""x"": 0.26875, ""y"": 0.08055555555555556, ""w"": 0.0734375, ""h"": 0.5152777777777777}, ""centroid"": {""x"": 391, ""y"": 244}, ""centroid_norm"": {""x"": 0.30546875, ""y"": 0.33819444444444446}, ""track_id"": 1}]}",cam_01,store_01,2025-11-06 09:52:52.495000+00:00
3,1.0,27f0344f42da49558518258a3d50263c,4,"{""schema_version"": ""1.0"", ""pipeline_run_id"": ""27f0344f42da49558518258a3d50263c"", ""source"": {""store_id"": ""store_01"", ""camera_id"": ""cam_01"", ""stream_id"": ""stream_01""}, ""frame_index"": 4, ""capture_ts"": ""2025-10-31T13:22:36.789379+00:00"", ""image_size"": {""width"": 1280, ""height"": 720}, ""detections"": [{""det_id"": ""4-0"", ""class"": null, ""class_id"": 0, ""conf"": 0.8237, ""bbox"": {""x1"": 344.0, ""y1"": 57.0, ""x2"": 437.0, ""y2"": 430.0}, ""bbox_norm"": {""x"": 0.26875, ""y"": 0.07916666666666666, ""w"": 0.07265625, ""h"": 0.5180555555555556}, ""centroid"": {""x"": 390, ""y"": 244}, ""centroid_norm"": {""x"": 0.305078125, ""y"": 0.33819444444444446}, ""track_id"": 1}]}",cam_01,store_01,2025-11-06 09:52:52.496000+00:00
4,1.0,27f0344f42da49558518258a3d50263c,5,"{""schema_version"": ""1.0"", ""pipeline_run_id"": ""27f0344f42da49558518258a3d50263c"", ""source"": {""store_id"": ""store_01"", ""camera_id"": ""cam_01"", ""stream_id"": ""stream_01""}, ""frame_index"": 5, ""capture_ts"": ""2025-10-31T13:22:36.829484+00:00"", ""image_size"": {""width"": 1280, ""height"": 720}, ""detections"": [{""det_id"": ""5-0"", ""class"": null, ""class_id"": 0, ""conf"": 0.8251, ""bbox"": {""x1"": 344.0, ""y1"": 58.0, ""x2"": 437.0, ""y2"": 431.0}, ""bbox_norm"": {""x"": 0.26875, ""y"": 0.08055555555555556, ""w"": 0.07265625, ""h"": 0.5180555555555556}, ""centroid"": {""x"": 390, ""y"": 244}, ""centroid_norm"": {""x"": 0.305078125,

In [9]:
# Lấy payload của 1 dòng
cursor.execute("SELECT payload FROM bronze_raw WHERE payload IS NOT NULL LIMIT 1")
result = cursor.fetchone()

if result:
    raw_payload_str = result[0]
    
    # Parse (phân tích) chuỗi JSON bằng Python
    payload_json = json.loads(raw_payload_str)
    
    # In ra cấu trúc JSON cho dễ đọc
    print("Cấu trúc của 1 dòng dữ liệu 'payload':")
    print(json.dumps(payload_json, indent=2, ensure_ascii=False))
else:
    print("Không tìm thấy dòng nào có payload. (Có thể job Flink chưa ghi dữ liệu?)")

Cấu trúc của 1 dòng dữ liệu 'payload':
{
  "schema_version": "1.0",
  "pipeline_run_id": "27f0344f42da49558518258a3d50263c",
  "source": {
    "store_id": "store_01",
    "camera_id": "cam_01",
    "stream_id": "stream_01"
  },
  "frame_index": 1,
  "capture_ts": "2025-10-31T13:22:36.674047+00:00",
  "image_size": {
    "width": 1280,
    "height": 720
  },
  "detections": [
    {
      "det_id": "1-0",
      "class": null,
      "class_id": 0,
      "conf": 0.8199,
      "bbox": {
        "x1": 345.0,
        "y1": 58.0,
        "x2": 438.0,
        "y2": 430.0
      },
      "bbox_norm": {
        "x": 0.26953125,
        "y": 0.08055555555555556,
        "w": 0.07265625,
        "h": 0.5166666666666667
      },
      "centroid": {
        "x": 392,
        "y": 244
      },
      "centroid_norm": {
        "x": 0.305859375,
        "y": 0.3388888888888889
      },
      "track_id": null
    }
  ]
}


In [10]:
# Đếm số dòng (records) theo store_id và camera_id
df_counts = pd.read_sql("""
    SELECT 
        store_id, 
        camera_id, 
        COUNT(*) AS total_records
    FROM bronze_raw
    GROUP BY 1, 2
    ORDER BY 3 DESC
""", conn)

print("Phân bố dữ liệu theo camera/store:")
df_counts

/tmp/ipykernel_58481/1412639990.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_counts = pd.read_sql("""


Phân bố dữ liệu theo camera/store:


,store_id,camera_id,total_records
0,store_01,cam_01,410


In [1]:
!pip install sqlalchemy trino[sqlalchemy]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 17.4 MB/s  0:00:00.8 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 607.6/607.6 kB 56.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [sqlalchemy] 1/2 [sqlalchemy]

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [6]:
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine("trino://python-client@localhost:8080/lakehouse/rva")

sql = """
SELECT record_count, file_size_in_bytes, file_path
FROM "bronze_raw$files"
ORDER BY file_size_in_bytes DESC
LIMIT 10
"""
df = pd.read_sql(sql, engine)

